# Thread Mapping and GPU Occupancy
In this section we will cover the how SYCL thread mapping works and how to achieve maximum occupancy with a kernel:
- [SYCL Thread Mapping](#SYCL-Thread-Mapping)
- [Mapping Work-groups to Xe-cores for Maximum Occupancy](#Mapping-Work-groups-to-Xe-cores-for-Maximum-Occupancy)
- [Thread Synchronization](#Thread-Synchronization)
- [GPU Occupancy Calculation Examples](#GPU-Occupancy-Calculation-Examples)
- [Intel® GPU Occupancy Calculator](#Intel®-GPU-Occupancy-Calculator)

## SYCL Thread Mapping
The SYCL* execution model exposes an abstract view of GPU execution. The SYCL thread hierarchy consists of a 1-, 2-, or 3-dimensional grid of work-items. These work-items are grouped into equal sized thread groups called work-groups. Threads in a work-group are further divided into equal sized vector groups called sub-groups (see the illustration that follows).
#### Work-item
A work-item represents one of a collection of parallel executions of a kernel.
#### Sub-group
A sub-group represents a short range of consecutive work-items that are processed together as a SIMD vector of length 8, 16, 32, or a multiple of the native vector length of a CPU with Intel® UHD Graphics.
#### Work-group
A work-group is a 1-, 2-, or 3-dimensional set of threads within the thread hierarchy. In SYCL, synchronization across work-items is only possible with barriers for the work-items within the same work-group.
### nd_range
An nd_range divides the thread hierarchy into 1-, 2-, or 3-dimensional grids of work-groups. It is represented by the global range, the local range of each work-group.

Thread hierarchy:
<img src="assets/nd_range.png">

The diagram above illustrates the relationship among ND-Range, work-group, sub-group, and work-item.

## Mapping Work-groups to Xe-cores for Maximum Occupancy
The rest of this chapter explains how to pick a proper work-group size to maximize the occupancy of the GPU resources. The new terminologies are Xe-core (XC) for Subslice, and Xe Vector Engine (XVE) for Execution Unit(EU).

From the key [GPU architecture parameters](https://www.intel.com/content/www/us/en/develop/documentation/oneapi-gpu-optimization-guide/top/xe-arch.html), we summarize the architecture parameters for Gen9, Gen11, Gen12 and Xe-HPC below:

Generations | Threads per XVE | XVEs per XC | Threads per XC | XCs | Total XVEs | Total Threads | Max Work Group Size
---|:-:|:-:|:-:|:-:|:-:|:-:|:-:
Intel UHD Graphics P630 SKL (Gen9) | 7 | 8  | 56  | 3 | 24 | 168 | 256
Intel Iris Xe ICL (Gen11)      | 7 | 8  | 56  | 8 | 64 | 448 | 256
Intel Iris Xe-LP TGL (Gen12)   | 7 | 16 | 112 | 6 | 96 | 672 | 512
Intel DataCenter Max 1100 PVC (Xe HPC)| 8 | 8  |  64 | 56 | 448 | 3584 | 1024
Intel DataCenter Max 1550 PVC (Xe HPC)| 8 | 8  |  64 | 128 | 1024 | 8192 | 1024

<img src="assets/xe_core.png">

<img src="assets/xe_stack_2.png">

The maximum work-group size is a constraint imposed by the hardware and GPU driver. You can query the maximum work-group using `device::get_info<sycl::info::device::max_work_group_size>()` on the supported size.

Let’s start with a simple kernel:
```cpp
auto command_group = [&](auto &cgh) {
      cgh.parallel_for(sycl::range<3>(112, 120, 128), // global range
                       [=](item<3> it) {
                         // (kernel code)
      })
}
```
This kernel contains 1,720,320 work-items structured as a 3D range of 112 x 120 x 128. It leaves the work-group and sub-group size selection to the compiler. To fully utilize the all parallel operations available in the GPU, the compiler must choose a proper work group size.

The two most important GPU resources are:
- __Thread Contexts:__
The kernel should have a sufficient number of threads to utilize the GPU’s thread contexts.
- __SIMD Units and SIMD Registers:__
The kernel should be organized to vectorize the work-items and utilize the SIMD registers.

In a SYCL kernel, the programmer can affect the work distribution by structuring the kernel with proper work-group size, sub-group size, and organizing the work-items for efficient vector execution. Writing efficient vector kernels is covered in a separate section. This chapter focuses on work-group and sub-group size selection.

SYCL does not provide a mechanism to directly set the number of threads in a work-group. However, you can use work-group size and SIMD sub-group size to set the number of threads:
```
Work group size = Threads x SIMD sub-group size
```
You can increase the sub-group size as long as there are a sufficient number of registers for the kernel after widening. In SYCL, a programmer can explicitly specify sub-group size using `intel::reqd_sub_group_size({16|32})` to override the compiler’s selection.

#### Query GPU info

The code below creates a SYCL queue with default device selector and queries for `max_work_group_size`, `sub_group_sizes` and `local_mem_size`. This information can be useful to know and choose appropriate work-group, sub-group and local-memory sizes in kernel code.

In [ ]:
%%writefile lab/gpu_support.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main() {
  sycl::queue q;
   
  //# Query for device information
  auto device_name = q.get_device().get_info<sycl::info::device::name>();
  auto wg_size = q.get_device().get_info<sycl::info::device::max_work_group_size>();
  auto sg_sizes = q.get_device().get_info<sycl::info::device::sub_group_sizes>();
  auto slm_size = q.get_device().get_info<sycl::info::device::local_mem_size>();
    
  std::cout << "Device : " << device_name << "\n";

  std::cout << "Max Work-Group Size : " << wg_size << "\n";

  std::cout << "Supported Sub-Group Sizes : ";
  for (int i=0; i<sg_sizes.size(); i++) std::cout << sg_sizes[i] << " "; std::cout << "\n";

  std::cout << "Local Memory Size : " << slm_size << "\n";
   
  q.submit([&](sycl::handler &h){
    h.parallel_for(sycl::nd_range<3>(sycl::range<3>(112, 120, 128), sycl::range<3>(1, 1, 128)), [=](sycl::nd_item<3> item)[[intel::reqd_sub_group_size(32)]] {
     // Kernel Code
    });
  }).wait();
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! ./q.sh run_gpu_support.sh

## Thread Synchronization
SYCL provides two synchronization mechanisms that can be called within a kernel function. Both are only defined for work-items within the same work-group. SYCL does not provide any global synchronization mechanism inside a kernel for all work-items across the entire nd_range.

- __group_barrier__ inserts a memory fence and blocks the execution of all work-items within the work-group until all work-items have reached its location.

- __mem_fence__ inserts a memory fence on global and local memory access across all work-items in a work-group.


### Impact of Work-item Synchronization Within Work-group
Let’s look at a kernel requiring work-item synchronization:
```cpp
auto command_group = [&](auto &cgh) {
      cgh.parallel_for(sycl::nd_range(sycl::range(112, 120, 128), // global range
                                sycl::range(1, R, 128)    // local range
                                ),
                       [=](sycl::nd_item<3> item) {
                         // (kernel code)
                         // Internal synchronization
                         sycl::group_barrier(item.get_group());
                         // (kernel code)
      });
}
```

This kernel is similar to the previous example, except it requires work-group barrier synchronization. Work-item synchronization is only available to work-items within the same work-group. You must pick a work-group local range using nd_range and nd_item. All the work-items of a work-group must be allocated to the same Xe-core, which affects Xe-core occupancy and kernel performance.

#### Xe-core Utilization for various configurations

The table below summarizes the tradeoffs between work-group size, number of threads, Xe-core utilization, and occupancy for __Intel Data Center GPU MAX__

      Total Work-Items = (112, 120, 128)

      Work-Group Size  = (1, R, 128)

      Sub-Group Size = 32

Work-Group Size | Threads | Xe-core Utilization | Xe-core Occupancy
--- | --- | --- | ---
(1, 1, 128) = 128 | 128/32 = 4 | 4/64 = 6.25% | 100% with 16 work-groups
(1, 2, 128) = 256 | 256/32 = 8| 8/64 = 12.5% | 100% with 8 work-groups
(1, 3, 128) = 384 | 384/32 = 12 | 12/64 = 18.75% | 93.75% with 5 work-groups
(1, 4, 128) = 512 | 512/32 = 16 | 16/64 = 25% | 100% with 4 work-groups
(1, 5, 128) = 640 | 640/32 = 20 | 20/64 = 31.25% | 93.75% with 3 work-groups
(1, 6, 128) = 768 | 768/32 = 24 | 24/64 = 37.5% | 75% with 2 work-groups
(1, 7, 128) = 896 | 896/32 = 28 | 28/64 = 44.75% | 89.5% with 2 work-groups
(1, 8, 128) = 1024 | 1024/32 = 32 | 32/64 = 50% | 100% with 2 work-groups
(1, R>8, 128) = 1024+ |  |  | Fail to launch


In this kernel, the local range of work-group is given as range(1, R, 128). Assuming the sub-group size is 32, let’s look at how the values of variable R affect Xe-core occupancy. In the case of R=1, the local group range is (1, 1, 128) and work-group size is 128. The Xe-core allocated for a work-group contains only 4 threads out of 64 available thread contexts (i.e., very low occupancy). However, the system can dispatch 16 work-groups to the same Xe-core to reach full occupancy at the expense of a higher number of dispatches.

In the case of R>8, the work-group size will exceed the system-supported maximum work-group size of 1024, and the kernel will fail to launch. In the case of R=6, an Xe-core is only 75% occupied. Note that the driver may still be able to dispatch a partial work-group to an unused Xe-core. However, because of the barrier in the kernel, the partially dispatched work items would not be able to pass the barriers until the rest of the work group is dispatched. In most cases, the kernel’s performance would not benefit much from the partial dispatch. Hence, it is important to avoid this problem by properly choosing the work-group size.

### Impact of Local Memory Within Work-group
Let’s look at an example where a kernel allocates local memory for a work-group:
```cpp
auto command_group =
    [&](auto &cgh) {
      // local memory variables shared among work items
      sycl::local_accessor<int, 1> myLocal(sycl::range(R), cgh);
      cgh.parallel_for(sycl::nd_range(sycl::range<3>(112, 120, 128), // global range
                                sycl::range<3>(1, R, 128)    // local range
                                ),
                       [=](sycl::nd_item<3> item) {
                         // (work group code)
                         myLocal[item.get_local_id()[0]] = ...
                       });
    }
```
Because work-group local variables are shared among its work-items, they are allocated in a Xe-core’s SLM. Therefore, this work-group must be allocated to a single Xe-core, same as the intra-group synchronization. In addition, you must also weigh the sizes of local variables under different group size options such that the local variables fit within an Xe-core’s 128KB SLM capacity limit.

## Intel® GPU Occupancy Calculator
In summary, a SYCL work-group is typically dispatched to an Xe-core. All the work-items in a work-group shares the same SLM of an Xe-core for intra work-group thread barriers and memory fence synchronization. Multiple work-groups can be dispatched to the same Xe-core if there are sufficient XVE ALUs, SLM, and thread contexts to accommodate them.

You can achieve higher performance by fully utilizing all available Xe-cores. Parameters affecting a kernel’s GPU occupancy are work-group size and SIMD sub-group size, which also determines the number of threads in the work-group.

The [Intel® GPU Occupancy Calculator](https://oneapi-src.github.io/oneAPI-samples/Tools/GPU-Occupancy-Calculator/index.html) can be used to calculate the occupancy on an Intel GPU for a given kernel, and its work-group parameters.

## GPU Occupancy Calculation Examples
Before concluding this section, let’s look at the hardware occupancies from the variants of a simple vector add example.

### Intel(R) UHD Graphics P630 (Gen9 GPU)

GPU | Threads per XVE | XVEs per XC | Threads per XC | XCs | Total XVEs | Total Threads | Max Work Group Size
---|:-:|:-:|:-:|:-:|:-:|:-:|:-:
Intel UHD Graphics P630 SKL (Gen9) | 7 | 8  | 56  | 3 | 24 | 168 | 256

The _vec_add_ example below explicitly specifies the work-group size to `max_work_group_size` (256), SIMD width of 32, and a variable number of work-groups as a function parameter groups.

In the absence of intra-work group synchronization, threads from any work-group can be dispatched to any Xe-core. Dividing the number of threads by the number of available thread contexts in the GPU (168) gives us an estimate of the GPU hardware occupancy. 

```
GPU Occupancy = (Work-Groups * (WG-Size/SIMD)) / GPU-Threads

GPU Occupancy = (Work-Groups * (256/32)) / 168

```

| Work-groups | Work-items | Work-group Size | SIMD | Threads work-group | Threads | XeCore Occupancy<br> = Threads / 56 |GPU Occupancy<br> = Threads/168 |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 1  | 256  | 256 | 32 | 8 | 8   | 14.2% | 4.7% |
| 2  | 512  | 256 | 32 | 8 | 16  | 28.5% | 9.5% |
| 3  | 768  | 256 | 32 | 8 | 24  | 42.8% | 14.2% |
| 4  | 1024 | 256 | 32 | 8 | 32  | 57.1% | 19% |
| 5  | 1280 | 256 | 32 | 8 | 40  | 71.4% | 23.8% |
| 6  | 1536 | 256 | 32 | 8 | 48  | 85.7% | 28.5% |
| 7  | 1792 | 256 | 32 | 8 | 56  | 100%  | 33.3% |
| 8  | 2048 | 256 | 32 | 8 | 64  | 100%  | 38% |
| 12 | 3072 | 256 | 32 | 8 | 96  | 100%  | 57% |
| 16 | 4096 | 256 | 32 | 8 | 128 | 100%  | 76% |
| 20 | 5120 | 256 | 32 | 8 | 160 | 100%  | 95% |
| 24 | 6144 | 256 | 32 | 8 | 192 | 100%  | 100% |

### Intel(R) Data Center GPU MAX 1100 (Xe HPC)

GPU | Threads per XVE | XVEs per XC | Threads per XC | XCs | Total XVEs | Total Threads | Max Work Group Size
---|:-:|:-:|:-:|:-:|:-:|:-:|:-:
Intel DataCenter Max 1100 PVC (Xe HPC)| 8 | 8  |  64 | 56 | 448 | 3584 | 1024

The _vec_add_ example below explicitly specifies the work-group size to `max_work_group_size` (1024), SIMD width of 32, and a variable number of work-groups as a function parameter groups.

In the absence of intra-work group synchronization, threads from any work-group can be dispatched to any Xe-core. Dividing the number of threads by the number of available thread contexts in the GPU (3584) gives us an estimate of the GPU hardware occupancy. 

```
GPU Occupancy = (Work-Groups * (WG-Size/SIMD)) / GPU-Threads

GPU Occupancy = (Work-Groups * (1024/32)) / 3584

```

| Work-groups | Work-items | Work-group Size | SIMD | Threads work-group | Threads | XeCore Occupancy<br> = Threads / 64 |GPU Occupancy<br> = Threads/3584 |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 1  | 1024  | 1024 | 32 | 32 | 32  | 50% | 0.9% |
| 2  | 2048  | 1024 | 32 | 32 | 64  | 100% | 1.7% |
| 8  | 8192 | 1024 | 32 | 32 | 256  | 100%  | 7.1% |
| 16 | 16384 | 1024 | 32 | 32 | 512 | 100%  | 14% |
| 24 | 24576 | 1024 | 32 | 32 | 768 | 100%  | 21% |
| 32 | 32768 | 1024 | 32 | 32 | 1024 | 100%  | 28% |
| 40 | 40690 | 1024 | 32 | 32 | 1280 | 100%  | 35% |
| 48 | 49152 | 1024 | 32 | 32 | 1536 | 100%  | 43% |
| 56 | 57344 | 1024 | 32 | 32 | 1792 | 100%  | 50% |
| 64 | 65536 | 1024 | 32 | 32 | 2048 | 100%  | 57% |
| 72 | 73728 | 1024 | 32 | 32 | 2304 | 100%  | 64% |
| 80 | 81920 | 1024 | 32 | 32 | 2560 | 100%  | 71% |
| 88 | 90112 | 1024 | 32 | 32 | 2816 | 100%  | 78% |
| 96 | 98304 | 1024 | 32 | 32 | 3072 | 100%  | 85% |
| 104 | 106496 | 1024 | 32 | 32 | 3328 | 100%  | 92% |
| 112 | 114688 | 1024 | 32 | 32 | 3584 | 100%  | 100% |
| 120 | 122880 | 1024 | 32 | 32 | 3840 | 100%  | 100% |
| 128 | 131072 | 1024 | 32 | 32 | 4096 | 100%  | 100% |
| 256 | 262144 | 1024 | 32 | 32 | 8192 | 100%  | 100% |
| 384 | 393216 | 1024 | 32 | 32 | 12288 | 100%  | 100% |
| 512 | 524288 | 1024 | 32 | 32 | 16384 | 100%  | 100% |
| 1024 | 1048576 | 1024 | 32 | 32 | 32768 | 100%  | 100% |


The `vec_add.cpp` SYCL code example below can be used to run and capture VTune Profiling data to 

In [ ]:
%%writefile lab/vec_add.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

#define N 13762560
const int WG_SIZE = 256; // 256,512,1024
const int SG_SIZE = 32;  // 8, 16, 32

template <int groups, int wg_size, int sg_size>
int VectorAdd(sycl::queue &q, std::vector<int> &a, std::vector<int> &b,
               std::vector<int> &sum) {
  sycl::range num_items{a.size()};

  sycl::buffer a_buf(a);
  sycl::buffer b_buf(b);
  sycl::buffer sum_buf(sum.data(), num_items);
  size_t num_groups = groups;

  auto start = std::chrono::high_resolution_clock::now().time_since_epoch().count();
  q.submit([&](auto &h) {
    sycl::accessor a_acc(a_buf, h, sycl::read_only);
    sycl::accessor b_acc(b_buf, h, sycl::read_only);
    sycl::accessor sum_acc(sum_buf, h, sycl::write_only, sycl::no_init);

    h.parallel_for(
        sycl::nd_range<1>(num_groups * wg_size, wg_size), [=
    ](sycl::nd_item<1> index) [[intel::reqd_sub_group_size(sg_size)]] {
          size_t grp_id = index.get_group()[0];
          size_t loc_id = index.get_local_id();
          size_t start = grp_id * N;
          size_t end = start + N;
          for (size_t i = start + loc_id; i < end; i += wg_size) {
            sum_acc[i] = a_acc[i] + b_acc[i];
          }
        });
  });
  q.wait();
  auto end = std::chrono::high_resolution_clock::now().time_since_epoch().count();
  std::cout << "VectorAdd<" << groups << "> completed on device - "
            << (end - start) * 1e-9 << " seconds\n";
  return 0;
}

int main() {

  sycl::queue q;

  std::vector<int> a(N), b(N), sum(N);
  for (size_t i = 0; i < a.size(); i++){
    a[i] = i;
    b[i] = i;
    sum[i] = 0;
  }

  std::cout << "Running on device: "
            << q.get_device().get_info<sycl::info::device::name>() << "\n";
  std::cout << "Vector size: " << a.size() << "\n";
    
  VectorAdd<1,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<2,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<3,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<4,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<5,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<6,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<7,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<8,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<12,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<16,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<20,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<24,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<28,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<32,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<40,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<48,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<56,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<64,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<80,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<96,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<112,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<128,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<192,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<256,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<384,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<512,WG_SIZE,SG_SIZE>(q, a, b, sum);
  VectorAdd<1024,WG_SIZE,SG_SIZE>(q, a, b, sum);
    
  return 0;
}

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! ./q.sh run_vec_add.sh

#### VTune Analysis
You can collect VTune `gpu_hotspots` data and check VTune Profiling Data using the `vtune_collect.sh` script, this script will collect VTune data using command-line options, you can then open the results in the VTune Profiler tool.

You can run this command in terminal, login to a GPU nodeand run the script `source vtune_collect.sh`, collecting vtune data will take few minutes and will generate a html report in the same folder, it will also create a folder `vtune_data` folder, this folder can be zipped up and copied to local machine with VTune GUI tool installed to analyze further like shown below:

From the VTune output below you can see that Occupancy we predicted earlier doing the manual calculation and the VTune data are the same:

<img src="assets/vtune_tasks_vec_add_gen9.png">

The GPU Compute Threads Dispatch can be analyzed on VTune Profiler:

<img src="assets/vtune_threads_vec_add_gen9.png">

## Resources

- [Intel GPU Optimization Guide](https://www.intel.com/content/www/us/en/develop/documentation/oneapi-gpu-optimization-guide/top.html) - Up to date resources for Intel GPU Optimization
- [SYCL Specification](https://registry.khronos.org/SYCL/specs/sycl-2020/pdf/sycl-2020.pdf) - Latest Specification document for reference
- [SYCL Essentials Training](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/C%2B%2BSYCL/Jupyter/oneapi-essentials-training) - Learn basics of C++ SYCL Programming